<a href="https://colab.research.google.com/github/seungmin001/prac_deeplearningzerotoall/blob/main/Lab6_2_Fancy_softmax_classifier_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import tensorflow as tf

path='/content/drive/MyDrive/Colab Notebooks/deeplearning_zero_to_all/data-04-zoo.csv'

data=np.loadtxt(path, delimiter=',', dtype=np.float32)
x_data= data[:, 0:-1]
y_data= data[:, [-1]]

print(x_data.shape, y_data.shape)

nb_classes=7

(101, 16) (101, 1)


In [15]:
# Make y_data as one-hot shape

y_one_hot=tf.one_hot(y_data, nb_classes)
y_one_hot=tf.reshape(y_one_hot, [-1, nb_classes])

print(y_one_hot.shape)

(101, 7)


In [16]:
# Dataset 변환

dataset= tf.data.Dataset.from_tensor_slices((x_data,y_one_hot)) # x_data옆에 y_one_hot 붙어서 저장된 dataset 생성

dataset.shuffle(len(x_data)).batch(16).repeat(100)  # 처음부터 끝까지 섞고, 16개씩 묶고, 100번 반복늘림

dataset

<TensorSliceDataset shapes: ((16,), (7,)), types: (tf.float32, tf.float32)>

In [28]:
# Weight, bias
W = tf.Variable(tf.random.normal([ x_data.shape[1] , nb_classes])) # 동물 특징 16가지임
b = tf.Variable(tf.random.normal([ nb_classes ])) # label만큼..

#softmax
def logit_fn(X):    # return 식을 logit라고 함
    return tf.matmul(X,W)+b

def cost_fn(X,Y):
    logits = logit_fn(X)
    # fancy function
    cost_i = tf.nn.softmax_cross_entropy_with_logits(Y,logits)
    cost_mean = tf.reduce_mean(cost_i)
    return cost_mean

def grad_fn(X,Y):
    with tf.GradientTape() as tape:
        cost = cost_fn(X,Y)
        grads = tape.gradient(cost, [W,b])
        return grads


# 검증을 위한 함수
def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), axis=1)
    correct = tf.argmax(y_one_hot, axis=1)
    correct_pred = tf.equal(pred, correct) # element-wise 비교해서 각 항마다 True, false 적힘

    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    return accuracy


In [29]:
# Training

def fit(X, Y, epochs=100, verbose=50): # verbose 출력 언제할지 정하는거
    opt = tf.optimizers.SGD(learning_rate=0.1)

    for step in range(epochs):
        grads=grad_fn(X,Y)
        opt.apply_gradients(zip(grads, [W,b]))
        
        if (step==0) or ((step+1)%verbose==0) :
            acc = prediction(X,Y).numpy()
            loss = tf.reduce_sum(cost_fn(X,Y)).numpy()

            print("Acc & loss at {:5} epoch : {:10.4f} {:12.4f}".format(step+1, acc, loss))
         
fit(x_data, y_one_hot, epochs=1000, verbose=100)            

Acc & loss at     1 epoch :     0.1782       6.6252
Acc & loss at   100 epoch :     0.7921       0.6307
Acc & loss at   200 epoch :     0.9109       0.3686
Acc & loss at   300 epoch :     0.9208       0.2661
Acc & loss at   400 epoch :     0.9505       0.2098
Acc & loss at   500 epoch :     0.9703       0.1736
Acc & loss at   600 epoch :     0.9703       0.1481
Acc & loss at   700 epoch :     0.9802       0.1290
Acc & loss at   800 epoch :     0.9802       0.1143
Acc & loss at   900 epoch :     0.9901       0.1025
Acc & loss at  1000 epoch :     1.0000       0.0930
